# Model Persistence

In [2]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pathlib import Path
import pandas as pd

In [3]:
# Read in data
data = Path("../Resources/winequality.csv")
df = pd.read_csv(data, delimiter=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Prepare the data

In [4]:
# Define the features X set and the target y vector
X = df.iloc[:, 0:11].values
y = df["quality"].values

In [5]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

## Create the shallow neural network model

In [6]:
# Define the model - shallow neural network
nn = Sequential()

# Hidden layer
nn.add(Dense(units=8, input_dim=11, activation="relu"))

# Output layer
nn.add(Dense(units=1, activation="linear"))

In [7]:
# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model = nn.fit(X, y, epochs=200, verbose=1) 

Epoch 1/200
50/50 [==============================] - 0s 650us/step - loss: 32.2293 - mse: 32.2293
Epoch 2/200
50/50 [==============================] - 0s 670us/step - loss: 27.8172 - mse: 27.8172
Epoch 3/200
50/50 [==============================] - 0s 735us/step - loss: 23.7588 - mse: 23.7588
Epoch 4/200
50/50 [==============================] - 0s 640us/step - loss: 19.9020 - mse: 19.9020
Epoch 5/200
50/50 [==============================] - 0s 616us/step - loss: 16.3214 - mse: 16.3214
Epoch 6/200
50/50 [==============================] - 0s 637us/step - loss: 13.1782 - mse: 13.1782
Epoch 7/200
50/50 [==============================] - 0s 670us/step - loss: 10.6558 - mse: 10.6558
Epoch 8/200
50/50 [==============================] - 0s 657us/step - loss: 8.7472 - mse: 8.7472
Epoch 9/200
50/50 [==============================] - 0s 678us/step - loss: 7.3792 - mse: 7.3792
Epoch 10/200
50/50 [==============================] - 0s 648us/step - loss: 6.4165 - mse: 6.4165
Epoch 11/200
50/50 [=====

## Save the model as JSON to a file

In [11]:
import json
# Save model as JSON
nn_json = nn.to_json()

file_path = Path("../Resources/model.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

# with open(file_path, "r") as json_file:
#     json_file = json.dumps(json_file)

## Save the model weights

In [8]:
# Save weights
file_path = "../Resources/model.h5"
nn.save_weights("../Resources/model.h5")

## Load the model and weights

In [13]:
# Load the saved model to make predictions

# Imports
from tensorflow.keras.models import model_from_json

# load json and create model
file_path = Path("../Resources/model.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = "../Resources/model.h5"
loaded_model.load_weights(file_path)

## Make predictions with the model

In [14]:
# Make some predictions with the loaded model
df["predicted"] = loaded_model.predict(X)
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,predicted
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4.986772
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,4.811093
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,5.426685
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,5.645499
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4.986772
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,5.045991
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,5.074284
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,5.179195
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,5.368835
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,5.299043


In [19]:
one_row = df.iloc[0, 0:11].values

X_one_row = scaler.transform([one_row]) # one row is not a 2D array. List around 1D array.

loaded_model.predict(X_one_row)[0][0]

4.9867716

In [42]:
# Weird one liner. DF here would be a DF of new data that matches the old!
# You are using the scaling you used in training (that is what the model was trainined on).
display(df.iloc[0,0:11])
loaded_model.predict(scaler.transform([df.iloc[0, 0:11].values]))[0][0]

fixed acidity            7.4000
volatile acidity         0.7000
citric acid              0.0000
residual sugar           1.9000
chlorides                0.0760
free sulfur dioxide     11.0000
total sulfur dioxide    34.0000
density                  0.9978
pH                       3.5100
sulphates                0.5600
alcohol                  9.4000
Name: 0, dtype: float64

4.9867716